# 第 1 阶段：基础 Agent + 工具调用

本 Notebook 用于学习和调试第 1 阶段的核心功能：
- 配置加载与验证
- LLM 模型创建
- 工具调用（时间、计算器、网络搜索等）
- Agent 基本功能（同步/异步、流式输出）

**参考文档**: `docs/stage_01/`  
**参考脚本**: `scripts/test_basic.py`

## 1. 环境准备

In [1]:
# 添加项目根目录到 Python 路径
import sys
from pathlib import Path

# 获取 backend 目录路径
backend_dir = Path.cwd()
if backend_dir.name == 'notebooks':
    backend_dir = backend_dir.parent

if str(backend_dir) not in sys.path:
    sys.path.insert(0, str(backend_dir))

print(f"✅ 项目根目录: {backend_dir}")

✅ 项目根目录: /Users/yuan/dev/ai-projects/lc-studylab/backend


## 2. 配置加载与验证

In [2]:
from config import settings, get_logger

# 初始化日志
logger = get_logger(__name__)

print("=" * 60)
print("配置信息")
print("=" * 60)
print(f"应用名称: {settings.app_name}")
print(f"版本: {settings.app_version}")
print(f"OpenAI 模型: {settings.openai_model}")
print(f"API Base: {settings.openai_api_base}")
print(f"温度参数: {settings.openai_temperature}")
print(f"日志级别: {settings.log_level}")
print("=" * 60)

2025-12-04 12:43:50.557 | INFO     | config.logging:setup_logging:83 | 📝 日志系统初始化完成 - 级别: INFO, 文件: logs/app.log


配置信息
应用名称: LC-StudyLab
版本: 0.1.0
OpenAI 模型: Qwen/Qwen3-14B
API Base: https://api.siliconflow.cn/v1/
温度参数: 0.7
日志级别: INFO


In [3]:
# 验证必需配置
try:
    settings.validate_required_keys()
    print("✅ 配置验证通过！")
except Exception as e:
    print(f"❌ 配置验证失败: {e}")
    print("\n请检查 .env 文件，确保 OPENAI_API_KEY 已设置")

✅ 配置验证通过！


## 3. LLM 模型创建

In [4]:
from core.models import get_chat_model

# 创建默认模型
model = get_chat_model()

print(f"✅ 模型创建成功")
print(f"模型类型: {model.__class__.__name__}")
print(f"模型名称: {settings.openai_model}")

2025-12-04 12:44:10.564 | INFO     | core.models:get_chat_model:81 | 🤖 创建聊天模型: Qwen/Qwen3-14B (temperature=0.7, streaming=True)


✅ 模型创建成功
模型类型: ChatOpenAI
模型名称: Qwen/Qwen3-14B


In [5]:
# 测试简单调用
response = model.invoke("你好，请用一句话介绍你自己")
print("\n模型响应:")
print(response.content)


模型响应:
我是通义千问，是阿里巴巴集团旗下的通义实验室研发的超大规模语言模型，我能够回答各种问题，提供信息和帮助。


## 4. 工具调用测试

### 4.1 时间工具

In [6]:
from core.tools import get_current_time, get_current_date

# 测试时间工具
time_result = get_current_time.invoke({})
date_result = get_current_date.invoke({})

print(f"⏰ 当前时间: {time_result}")
print(f"📅 当前日期: {date_result}")

2025-12-04 12:44:44.910 | WARNING  | core.tools.web_search:<module>:28 | ⚠️ 使用旧的 langchain-community Tavily（已弃用），建议安装: pip install langchain-tavily


⏰ 当前时间: 当前时间是：2025-12-04 12:44:44
📅 当前日期: 今天是：2025-12-04 (星期四)


### 4.2 计算器工具

In [8]:
from core.tools import calculator

# 测试计算器
test_expressions = [
    "2 + 2",
    "10 * 5 + 3",
    "(100 - 20) / 4",
    "2 ** 10"  # 2的10次方
]

print("🧮 计算器测试:")
for expr in test_expressions:
    result = calculator.invoke({"expression": expr})
    print(f"  {expr} = {result}")

🧮 计算器测试:
  2 + 2 = 2 + 2 = 4
  10 * 5 + 3 = 10 * 5 + 3 = 53
  (100 - 20) / 4 = (100 - 20) / 4 = 20
  2 ** 10 = 2 ** 10 = 1024


### 4.3 网络搜索工具（可选）

In [9]:
from core.tools import web_search_simple

# 检查是否配置了 Tavily API Key
if settings.tavily_api_key:
    print("🔍 测试网络搜索...")
    search_result = web_search_simple.invoke({"query": "LangChain 是什么"})
    print(f"\n搜索结果:\n{search_result}")
else:
    print("⚠️  未配置 TAVILY_API_KEY，跳过网络搜索测试")
    print("   如需使用，请在 .env 中设置 TAVILY_API_KEY")

⚠️  未配置 TAVILY_API_KEY，跳过网络搜索测试
   如需使用，请在 .env 中设置 TAVILY_API_KEY


### 4.4 查看所有可用工具

In [10]:
from core.tools import ALL_TOOLS, BASIC_TOOLS

print(f"📦 基础工具数量: {len(BASIC_TOOLS)}")
print(f"📦 所有工具数量: {len(ALL_TOOLS)}")

print("\n基础工具列表:")
for tool in BASIC_TOOLS:
    print(f"  - {tool.name}: {tool.description}")

📦 基础工具数量: 3
📦 所有工具数量: 8

基础工具列表:
  - get_current_time: 获取当前时间

返回格式化的当前日期和时间，格式为：YYYY-MM-DD HH:MM:SS

**注意：查询天气时不需要调用此工具！天气工具内部已经知道当前日期。**

Returns:
    当前时间的字符串表示

Example:
    >>> get_current_time()
    '2025-11-05 14:30:25'

使用场景：
    - 当用户明确问"现在几点"、"当前时间"时使用
    - **不要**在查询天气前调用此工具
  - get_current_date: 获取当前日期

返回格式化的当前日期，格式为：YYYY-MM-DD，以及星期几

**注意：查询天气时不需要调用此工具！天气工具内部已经知道当前日期。**

Returns:
    当前日期的字符串表示

Example:
    >>> get_current_date()
    '2025-11-05 (星期三)'

使用场景：
    - 当用户明确问"今天是几号"、"今天星期几"时使用
    - **不要**在查询天气前调用此工具
  - calculator: 计算数学表达式

支持基本的数学运算：加法(+)、减法(-)、乘法(*)、除法(/)、括号()

Args:
    expression: 要计算的数学表达式，例如 "2 + 2" 或 "(10 + 5) * 3"

Returns:
    计算结果的字符串表示

Example:
    >>> calculator("2 + 2")
    '2 + 2 = 4'

    >>> calculator("(10 + 5) * 3")
    '(10 + 5) * 3 = 45'

    >>> calculator("10 / 3")
    '10 / 3 = 3.3333333333'


## 5. Agent 基本功能

### 5.1 创建基础 Agent

In [17]:
from agents import create_base_agent
from core.tools import ALL_TOOLS

# 创建不带工具的 Agent
agent_simple = create_base_agent(prompt_mode="default")
print("✅ 创建简单 Agent 成功（无工具）")

# 创建带工具的 Agent
agent_with_tools = create_base_agent(prompt_mode="default", tools=ALL_TOOLS)
print("✅ 创建 Agent 成功（带工具）")

2025-12-04 15:46:19.456 | INFO     | agents.base_agent:create_base_agent:481 | 🏭 创建 Base Agent (mode=default, debug=False)
2025-12-04 15:46:19.457 | INFO     | agents.base_agent:__init__:109 | 🤖 使用默认模型: openai:Qwen/Qwen3-14B
2025-12-04 15:46:19.458 | INFO     | agents.base_agent:__init__:123 | 🔧 使用基础工具集 (3 个工具)
2025-12-04 15:46:19.459 | INFO     | agents.base_agent:__init__:139 | 📝 使用带工具说明的系统提示词 (模式: default)
2025-12-04 15:46:19.461 | INFO     | agents.base_agent:__init__:155 | 🔨 创建 Agent（使用 LangChain V1.0.0 create_agent API）...
2025-12-04 15:46:19.472 | INFO     | agents.base_agent:__init__:167 | ✅ Agent 创建成功（CompiledStateGraph）
2025-12-04 15:46:19.474 | INFO     | agents.base_agent:create_base_agent:481 | 🏭 创建 Base Agent (mode=default, debug=False)
2025-12-04 15:46:19.474 | INFO     | agents.base_agent:__init__:109 | 🤖 使用默认模型: openai:Qwen/Qwen3-14B
2025-12-04 15:46:19.475 | INFO     | agents.base_agent:__init__:126 | 🔧 使用自定义工具集 (8 个工具)
2025-12-04 15:46:19.475 | INFO     | agents.base

✅ 创建简单 Agent 成功（无工具）
✅ 创建 Agent 成功（带工具）


### 5.2 简单对话测试

In [18]:
# 测试简单对话
response = agent_simple.invoke("请用一句话解释什么是机器学习")
print("💬 Agent 回答:")
print(response)

2025-12-04 15:48:30.765 | INFO     | agents.base_agent:invoke:202 | 🚀 执行 Agent 调用: 请用一句话解释什么是机器学习...
2025-12-04 15:48:35.149 | INFO     | agents.base_agent:invoke:235 | ✅ Agent 调用完成，输出长度: 41 字符


💬 Agent 回答:
机器学习是通过分析数据自动发现模式，并利用这些模式对未知数据进行预测或决策的过程。


### 5.3 工具调用测试

In [19]:
# 测试时间查询（需要调用工具）
response = agent_with_tools.invoke("现在几点了？")
print("⏰ Agent 回答（调用时间工具）:")
print(response)

2025-12-04 16:12:55.702 | INFO     | agents.base_agent:invoke:202 | 🚀 执行 Agent 调用: 现在几点了？...
2025-12-04 16:13:10.543 | INFO     | agents.base_agent:invoke:235 | ✅ Agent 调用完成，输出长度: 26 字符


⏰ Agent 回答（调用时间工具）:
当前时间是 2025年12月4日 16:13:08。


In [20]:
# 测试计算（需要调用计算器）
response = agent_with_tools.invoke("帮我计算 123 * 456 等于多少")
print("🧮 Agent 回答（调用计算器）:")
print(response)

2025-12-04 16:13:25.351 | INFO     | agents.base_agent:invoke:202 | 🚀 执行 Agent 调用: 帮我计算 123 * 456 等于多少...
2025-12-04 16:14:29.426 | INFO     | agents.base_agent:invoke:235 | ✅ Agent 调用完成，输出长度: 18 字符


🧮 Agent 回答（调用计算器）:
123乘以456的结果是56088。


### 5.4 流式输出测试

In [22]:
# 创建支持流式输出的 Agent
agent_streaming = create_base_agent(prompt_mode="default")

print("📡 流式输出测试:")
print("="*60)

# 流式输出
for chunk in agent_streaming.stream("讲一个关于编程的笑话"):
    print(chunk, end="", flush=True)

print("\n" + "="*60)

2025-12-04 16:22:07.656 | INFO     | agents.base_agent:create_base_agent:481 | 🏭 创建 Base Agent (mode=default, debug=False)
2025-12-04 16:22:07.657 | INFO     | agents.base_agent:__init__:109 | 🤖 使用默认模型: openai:Qwen/Qwen3-14B
2025-12-04 16:22:07.657 | INFO     | agents.base_agent:__init__:123 | 🔧 使用基础工具集 (3 个工具)
2025-12-04 16:22:07.658 | INFO     | agents.base_agent:__init__:139 | 📝 使用带工具说明的系统提示词 (模式: default)
2025-12-04 16:22:07.658 | INFO     | agents.base_agent:__init__:155 | 🔨 创建 Agent（使用 LangChain V1.0.0 create_agent API）...
2025-12-04 16:22:07.666 | INFO     | agents.base_agent:__init__:167 | ✅ Agent 创建成功（CompiledStateGraph）
2025-12-04 16:22:07.667 | INFO     | agents.base_agent:stream:278 | 🌊 执行 Agent 流式调用: 讲一个关于编程的笑话...


📡 流式输出测试:


好的，这里有一个经典的编程笑话：

**为什么程序员总是分不清万圣节（10月31日）和圣诞节（12月25日）？**  
因为对于程序员来说：  
**Oct 31 == Dec 25** 😄  

**解析**：  
- "Oct" 是八进制（Octal）的缩写，而 "Dec" 是十进制（Decimal）的缩写。  
- 在八进制中，**Oct 31** 等于 3×8 + 1 = **25**（十进制）。  
- 所以程序员会误以为 "Oct 31" 和 "Dec 25" 是同一个数字！  

这个笑话利用了编程中常见的进制转换梗，是不是很有趣？😎  
你还想听更多编程相关的冷知识或笑话吗？

2025-12-04 16:22:19.045 | INFO     | agents.base_agent:stream:315 | ✅ Agent 流式调用完成


## 6. 不同模式的 Agent

### 6.1 编程助手模式

In [ ]:
agent_coding = create_base_agent(prompt_mode="coding")

response = agent_coding.invoke("如何在 Python 中实现单例模式？")
print("👨‍💻 编程助手回答:")
print(response)

### 6.2 研究助手模式

In [ ]:
agent_research = create_base_agent(mode="research")

response = agent_research.invoke("简述深度学习的发展历程")
print("🔬 研究助手回答:")
print(response)

### 6.3 简洁模式

In [ ]:
agent_concise = create_base_agent(mode="concise")

response = agent_concise.invoke("什么是 Docker？")
print("⚡ 简洁模式回答:")
print(response)

### 6.4 详细模式

In [ ]:
agent_detailed = create_base_agent(mode="detailed")

response = agent_detailed.invoke("什么是 Docker？")
print("📖 详细模式回答:")
print(response)

## 7. 对话历史管理

In [ ]:
# 创建带历史记录的 Agent
agent = create_base_agent(mode="default")

# 第一轮对话
response1 = agent.invoke("我叫小明，我喜欢编程")
print("第一轮:")
print(response1)

# 第二轮对话（测试是否记住上下文）
response2 = agent.invoke("我叫什么名字？我喜欢什么？")
print("\n第二轮（测试记忆）:")
print(response2)

## 8. 总结

通过本 Notebook，你已经学习了：

✅ 配置加载与验证  
✅ LLM 模型创建与调用  
✅ 各种工具的使用（时间、计算器、搜索）  
✅ Agent 的创建与基本功能  
✅ 同步调用与流式输出  
✅ 不同模式的 Agent（编程、研究、简洁、详细）  
✅ 对话历史管理

**下一步**: 学习第 2 阶段 - RAG 知识库（`stage_02_rag.ipynb`）